# **11.5 Minibatch Stochastic Gradient Descent**

11.5.1 Vectorization and Caches

In [ ]:
%matplotlib inline
import numpy as np
import torch
from torch import nn
from d2l import torch as d2l

timer = d2l.Timer()
A = torch.zeros(256, 256)
B = torch.randn(256, 256)
C = torch.randn(256, 256)

In [ ]:
# Compute A = BC one element at a time
timer.start()
for i in range(256):
  for j in range(256):
    A[i, j] = torch.dot(B[i, :], C[:, j])
timer.stop()

In [ ]:
# Compute A = BC one column at a time
timer.start()
for j in range(256):
  A[:, j] = torch.mv(B, C[:, j])
timer.stop()

In [ ]:
# Compute A = BC in one go
timer.start()
A = torch.mm(B, C)
timer.stop()

# Multiply and add count as separate operations (fused in practice)
gigaflops = [2/i for i in timer.times]
print(f'performance in Gigaflops: element {gigaflops[0]:.3f}, '
    f'column {gigaflops[1]:.3f}, full {gigaflops[2]:.3f}')

11.5.2 Minibatches

In [ ]:
timer.start()
for j in range(0, 256, 64):
A[:, j:j+64] = torch.mm(B, C[:, j:j+64])
timer.stop()
print(f'performance in Gigaflops: block {2 / timer.times[3]:.3f}')

11.5.3 Reading the Dataset

In [ ]:
#@save
d2l.DATA_HUB['airfoil'] = (d2l.DATA_URL + 'airfoil_self_noise.dat',
    '76e5be1548fd8222e5074cf0faae75edff8cf93f')

#@save
def get_data_ch11(batch_size=10, n=1500):
  data = np.genfromtxt(d2l.download('airfoil'),
    dtype=np.float32, delimiter='\t')
  data = torch.from_numpy((data - data.mean(axis=0)) / data.std(axis=0))
  data_iter = d2l.load_array((data[:n, :-1], data[:n, -1]),
    batch_size, is_train=True)
  return data_iter, data.shape[1]-1

11.5.4 Implementation from Scratch


In [ ]:
def sgd(params, states, hyperparams):
  for p in params:
    p.data.sub_(hyperparams['lr'] * p.grad)
    p.grad.data.zero_()

In [ ]:
#@save
def train_ch11(trainer_fn, states, hyperparams, data_iter,
  feature_dim, num_epochs=2):
  # Initialization
  w = torch.normal(mean=0.0, std=0.01, size=(feature_dim, 1),
        requires_grad=True)
  b = torch.zeros((1), requires_grad=True)
  net, loss = lambda X: d2l.linreg(X, w, b), d2l.squared_loss
  # Train
  animator = d2l.Animator(xlabel='epoch', ylabel='loss',
      xlim=[0, num_epochs], ylim=[0.22, 0.35])
  n, timer = 0, d2l.Timer()
  for _ in range(num_epochs):
    for X, y in data_iter:
      l = loss(net(X), y).mean()
      l.backward()
      trainer_fn([w, b], states, hyperparams)
      n += X.shape[0]
      if n % 200 == 0:
        timer.stop()
        animator.add(n/X.shape[0]/len(data_iter),
            (d2l.evaluate_loss(net, data_iter, loss),))
        timer.start()
  print(f'loss: {animator.Y[0][-1]:.3f}, {timer.avg():.3f} sec/epoch')
  return timer.cumsum(), animator.Y[0]

In [ ]:
def train_sgd(lr, batch_size, num_epochs=2):
  data_iter, feature_dim = get_data_ch11(batch_size)
  return train_ch11(
    sgd, None, {'lr': lr}, data_iter, feature_dim, num_epochs)
  
gd_res = train_sgd(1, 1500, 10)

In [ ]:
sgd_res = train_sgd(0.005, 1)

In [ ]:
mini1_res = train_sgd(.4, 100)

In [ ]:
mini2_res = train_sgd(.05, 10)

In [ ]:
d2l.set_figsize([6, 3])
d2l.plot(*list(map(list, zip(gd_res, sgd_res, mini1_res, mini2_res))),
  'time (sec)', 'loss', xlim=[1e-2, 10],
  legend=['gd', 'sgd', 'batch size=100', 'batch size=10'])
d2l.plt.gca().set_xscale('log')

11.5.5 Concise Implementation

In [ ]:
#@save
def train_concise_ch11(trainer_fn, hyperparams, data_iter, num_epochs=4):
  # Initialization
  net = nn.Sequential(nn.Linear(5, 1))
  def init_weights(m):
    if type(m) == nn.Linear:
      torch.nn.init.normal_(m.weight, std=0.01)
  net.apply(init_weights)

  optimizer = trainer_fn(net.parameters(), **hyperparams)
  loss = nn.MSELoss(reduction='none')
  animator = d2l.Animator(xlabel='epoch', ylabel='loss',
          xlim=[0, num_epochs], ylim=[0.22, 0.35])

  n, timer = 0, d2l.Timer()
  for _ in range(num_epochs):
    for X, y in data_iter:
      optimizer.zero_grad()
      out = net(X)
      y = y.reshape(out.shape)
      l = loss(out, y)
      l.mean().backward()
      optimizer.step()
      n += X.shape[0]
      if n % 200 == 0:
        timer.stop()
        # `MSELoss` computes squared error without the 1/2 factor
        animator.add(n/X.shape[0]/len(data_iter),
            (d2l.evaluate_loss(net, data_iter, loss) / 2,))
        timer.start()
  print(f'loss: {animator.Y[0][-1]:.3f}, {timer.avg():.3f} sec/epoch')

In [ ]:
data_iter, _ = get_data_ch11(10)
trainer = torch.optim.SGD
train_concise_ch11(trainer, {'lr': 0.01}, data_iter)

# **11.6 Momentum**

11.6.1 Basics

In [ ]:
%matplotlib inline
import torch
from d2l import torch as d2l
eta = 0.4
def f_2d(x1, x2):
  return 0.1 * x1 ** 2 + 2 * x2 ** 2

In [ ]:
def gd_2d(x1, x2, s1, s2):
  return (x1 - eta * 0.2 * x1, x2 - eta * 4 * x2, 0, 0)
d2l.show_trace_2d(f_2d, d2l.train_2d(gd_2d))

In [ ]:
eta = 0.6
d2l.show_trace_2d(f_2d, d2l.train_2d(gd_2d))

In [ ]:
def momentum_2d(x1, x2, v1, v2):
  v1 = beta * v1 + 0.2 * x1
  v2 = beta * v2 + 4 * x2
  return x1 - eta * v1, x2 - eta * v2, v1, v2

eta, beta = 0.6, 0.5
d2l.show_trace_2d(f_2d, d2l.train_2d(momentum_2d))

In [ ]:
eta, beta = 0.6, 0.25
d2l.show_trace_2d(f_2d, d2l.train_2d(momentum_2d))

In [ ]:
d2l.set_figsize()
betas = [0.95, 0.9, 0.6, 0]
for beta in betas:
  x = torch.arange(40).detach().numpy()
  d2l.plt.plot(x, beta ** x, label=f'beta = {beta:.2f}')
d2l.plt.xlabel('time')
d2l.plt.legend();

11.6.2 Practical Experiments

In [ ]:
def init_momentum_states(feature_dim):
  v_w = torch.zeros((feature_dim, 1))
  v_b = torch.zeros(1)
  return (v_w, v_b)

In [ ]:
def sgd_momentum(params, states, hyperparams):
  for p, v in zip(params, states):
    with torch.no_grad():
      v[:] = hyperparams['momentum'] * v + p.grad
      p[:] -= hyperparams['lr'] * v
    p.grad.data.zero_()

In [ ]:
def train_momentum(lr, momentum, num_epochs=2):
  d2l.train_ch11(sgd_momentum, init_momentum_states(feature_dim),
    {'lr': lr, 'momentum': momentum}, data_iter,
    feature_dim, num_epochs)

data_iter, feature_dim = d2l.get_data_ch11(batch_size=10)
train_momentum(0.02, 0.5)

In [ ]:
train_momentum(0.01, 0.9)

In [ ]:
train_momentum(0.005, 0.9)

In [ ]:
trainer = torch.optim.SGD
d2l.train_concise_ch11(trainer, {'lr': 0.005, 'momentum': 0.9}, data_iter)

11.6.3 Theoretical Analysis

In [ ]:
lambdas = [0.1, 1, 10, 19]
eta = 0.1
d2l.set_figsize((6, 4))
for lam in lambdas:
  t = torch.arange(20).detach().numpy()
  d2l.plt.plot(t, (1 - eta * lam) ** t, label=f'lambda = {lam:.2f}')
d2l.plt.xlabel('time')
d2l.plt.legend();

# **11.7 Adagrad**

11.7.1 Sparse Features and Learning Rates

11.7.2 Preconditioning

11.7.3 The Algorithm

In [ ]:
def adagrad_2d(x1, x2, s1, s2):
  eps = 1e-6
  g1, g2 = 0.2 * x1, 4 * x2
  s1 += g1 ** 2
  s2 += g2 ** 2
  x1 -= eta / math.sqrt(s1 + eps) * g1
  x2 -= eta / math.sqrt(s2 + eps) * g2
  return x1, x2, s1, s2

def f_2d(x1, x2):
  return 0.1 * x1 ** 2 + 2 * x2 ** 2

eta = 0.4
d2l.show_trace_2d(f_2d, d2l.train_2d(adagrad_2d))

In [ ]:
eta = 2
d2l.show_trace_2d(f_2d, d2l.train_2d(adagrad_2d))

11.7.4 Implementation from Scratch

In [ ]:
def init_adagrad_states(feature_dim):
  s_w = torch.zeros((feature_dim, 1))
  s_b = torch.zeros(1)
  return (s_w, s_b)

def adagrad(params, states, hyperparams):
  eps = 1e-6
  for p, s in zip(params, states):
    with torch.no_grad():
      s[:] += torch.square(p.grad)
      p[:] -= hyperparams['lr'] * p.grad / torch.sqrt(s + eps)
    p.grad.data.zero_()

In [ ]:
data_iter, feature_dim = d2l.get_data_ch11(batch_size=10)
d2l.train_ch11(adagrad, init_adagrad_states(feature_dim),
    {'lr': 0.1}, data_iter, feature_dim);

11.7.5 Concise Implementation

In [ ]:
trainer = torch.optim.Adagrad
d2l.train_concise_ch11(trainer, {'lr': 0.1}, data_iter)

# **11.8 RMSProp**

11.8.1 The Algorithm

In [ ]:
d2l.set_figsize()
gammas = [0.95, 0.9, 0.8, 0.7]
for gamma in gammas:
  x = torch.arange(40).detach().numpy()
  d2l.plt.plot(x, (1-gamma) * gamma ** x, label=f'gamma = {gamma:.2f}')
d2l.plt.xlabel('time');

11.8.2 Implementation from Scratch

In [ ]:
def rmsprop_2d(x1, x2, s1, s2):
  g1, g2, eps = 0.2 * x1, 4 * x2, 1e-6
  s1 = gamma * s1 + (1 - gamma) * g1 ** 2
  s2 = gamma * s2 + (1 - gamma) * g2 ** 2
  x1 -= eta / math.sqrt(s1 + eps) * g1
  x2 -= eta / math.sqrt(s2 + eps) * g2
  return x1, x2, s1, s2

def f_2d(x1, x2):
  return 0.1 * x1 ** 2 + 2 * x2 ** 2

eta, gamma = 0.4, 0.9
d2l.show_trace_2d(f_2d, d2l.train_2d(rmsprop_2d))

In [ ]:
def init_rmsprop_states(feature_dim):
  s_w = torch.zeros((feature_dim, 1))
  s_b = torch.zeros(1)
  return (s_w, s_b)

In [ ]:
def rmsprop(params, states, hyperparams):
  gamma, eps = hyperparams['gamma'], 1e-6
  for p, s in zip(params, states):
    with torch.no_grad():
      s[:] = gamma * s + (1 - gamma) * torch.square(p.grad)
      p[:] -= hyperparams['lr'] * p.grad / torch.sqrt(s + eps)
    p.grad.data.zero_()

In [ ]:
data_iter, feature_dim = d2l.get_data_ch11(batch_size=10)
d2l.train_ch11(rmsprop, init_rmsprop_states(feature_dim),
    {'lr': 0.01, 'gamma': 0.9}, data_iter, feature_dim);

11.8.3 Concise Implementation

In [ ]:
trainer = torch.optim.RMSprop
d2l.train_concise_ch11(trainer, {'lr': 0.01, 'alpha': 0.9},
    data_iter)

# **11.9 Adadelta**

11.9.1 The Algorithm

11.9.2 Implementation

In [ ]:
%matplotlib inline
import torch
from d2l import torch as d2l
  def init_adadelta_states(feature_dim):
    s_w, s_b = torch.zeros((feature_dim, 1)), torch.zeros(1)
    delta_w, delta_b = torch.zeros((feature_dim, 1)), torch.zeros(1)
    return ((s_w, delta_w), (s_b, delta_b))
  
  def adadelta(params, states, hyperparams):
    rho, eps = hyperparams['rho'], 1e-5
    for p, (s, delta) in zip(params, states):
      with torch.no_grad():
      # In-place updates via [:]
        s[:] = rho * s + (1 - rho) * torch.square(p.grad)
        g = (torch.sqrt(delta + eps) / torch.sqrt(s + eps)) * p.grad
        p[:] -= g
        delta[:] = rho * delta + (1 - rho) * g * g
      p.grad.data.zero_()

In [ ]:
data_iter, feature_dim = d2l.get_data_ch11(batch_size=10)
d2l.train_ch11(adadelta, init_adadelta_states(feature_dim),
    {'rho': 0.9}, data_iter, feature_dim);

In [ ]:
trainer = torch.optim.Adadelta
d2l.train_concise_ch11(trainer, {'rho': 0.9}, data_iter)